In [1]:
# Install required packages
# !pip install ultralytics==8.0.238 opencv-python-headless==4.8.1.78
# !pip install torch==1.10.0 torchvision==0.11.0 -f https://download.pytorch.org/whl/cu113/torch_stable.html
# !pip install onnx onnxruntime tensorrt

import torch
import torch.nn as nn
import numpy as np
from ultralytics import YOLO
from ultralytics.engine.model import Model
from ultralytics.nn.tasks import DetectionModel
from ultralytics.utils.torch_utils import select_device
import cv2
import json
import os
from pathlib import Path
import yaml
from PIL import Image
from tqdm import tqdm
import shutil
from collections import defaultdict

# Check environment
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# Download official YOLOv8n model (smallest, best for Jetson)
!wget https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt
print("Downloaded YOLOv8n base model")

PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA device: NVIDIA GeForce RTX 3070 Ti
CUDA memory: 8.0 GB
Downloaded YOLOv8n base model


'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
class YOLOv8GroundRobotDetector(nn.Module):
    """
    Modified YOLOv8 for ground-level robot detection with orientation.
    Handles arbitrary number of robots (0 to N) in each frame.
    """
    
    def __init__(self, model_path='yolov8n.pt', num_classes=2):  # robot and goal_box
        super().__init__()
        
        # Load pretrained YOLOv8
        self.base_model = YOLO(model_path)
        self.model = self.base_model.model
        
        # Modify the detection head to include orientation
        # YOLOv8n has 3 detection heads for different scales
        # We'll modify each to output: [x, y, w, h, obj, cls1, cls2, sin(yaw), cos(yaw)]
        
        # Get the Detect module
        self.detect = self.model.model[-1]
        
        # Original YOLOv8n outputs: nc=80 classes -> 85 channels per anchor
        # Our output: 2 classes + 2 orientation values -> 9 channels per anchor
        # (4 bbox + 1 obj + 2 cls + 2 orientation)
        
        old_detect = self.detect
        in_channels = [ch.out_channels for ch in self.model.model[-2].m]
        
        # Create new detection heads with orientation output
        self.detect.cv3 = nn.ModuleList(
            nn.Sequential(
                nn.Conv2d(x, 256, 3, padding=1),
                nn.BatchNorm2d(256),
                nn.ReLU(inplace=True),
                nn.Conv2d(256, (4 + 1 + num_classes + 2) * old_detect.na, 1)
            ) for x in in_channels
        )
        
        # Initialize weights
        for m in self.detect.cv3:
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        return self.model(x)

class OrientationLoss(nn.Module):
    """
    Custom loss for orientation estimation.
    Uses sin/cos representation to handle angle discontinuity.
    """
    
    def __init__(self):
        super().__init__()
        
    def forward(self, pred_sin_cos, target_angle, mask):
        """
        pred_sin_cos: [batch, 2] predicted sin and cos values
        target_angle: [batch] ground truth angles in radians
        mask: [batch] binary mask for valid orientations (robots only)
        """
        if mask.sum() == 0:
            return torch.tensor(0.0, device=pred_sin_cos.device)
        
        # Normalize predictions to unit circle
        pred_norm = torch.nn.functional.normalize(pred_sin_cos[mask], p=2, dim=1)
        
        # Convert target angles to sin/cos
        target_sin = torch.sin(target_angle[mask]).unsqueeze(1)
        target_cos = torch.cos(target_angle[mask]).unsqueeze(1)
        target_sin_cos = torch.cat([target_sin, target_cos], dim=1)
        
        # L2 loss in sin/cos space
        loss = torch.nn.functional.mse_loss(pred_norm, target_sin_cos)
        
        return loss

In [13]:
class GroundViewDatasetPrep:
    """
    Prepare dataset specifically for ground-level robot detection.
    Handles variable number of robots per image.
    """
    
    def __init__(self, source_dir, output_dir):
        self.source_dir = Path(source_dir)
        self.output_dir = Path(output_dir)
        
        # Create directory structure
        for split in ['train', 'val', 'test']:
            (self.output_dir / split / 'images').mkdir(parents=True, exist_ok=True)
            (self.output_dir / split / 'labels').mkdir(parents=True, exist_ok=True)
    
    def analyze_dataset_statistics(self):
        """Analyze robot count distribution and spatial patterns"""
        stats = defaultdict(int)
        robot_counts = []
        
        for json_path in self.source_dir.glob("screenshot_*.json"):
            if '_debug' in json_path.name:
                continue
                
            with open(json_path) as f:
                data = json.load(f)
                
            # Handle both old and new JSON format
            if isinstance(data, dict) and 'pose_data' in data:
                pose_list = data['pose_data']
            else:
                pose_list = data
                
            robot_count = sum(1 for p in pose_list if 'robot' in p.get('robot_id', ''))
            robot_counts.append(robot_count)
            stats[f'{robot_count}_robots'] += 1
        
        print("Dataset Statistics:")
        print(f"Total images: {len(robot_counts)}")
        print(f"Robot count distribution:")
        for k, v in sorted(stats.items()):
            print(f"  {k}: {v} images ({v/len(robot_counts)*100:.1f}%)")
        
        return stats
    
    def process_dataset(self):
        """Process images with enhanced ground-view awareness"""
        
        # First analyze the dataset
        stats = self.analyze_dataset_statistics()
        
        # Get all valid images
        all_images = []
        for img_path in self.source_dir.glob("screenshot_*.png"):
            if '_debug' not in img_path.name:
                txt_path = img_path.with_suffix('.txt')
                json_path = img_path.with_suffix('.json')
                if txt_path.exists() and json_path.exists():
                    all_images.append(img_path)
        
        print(f"\nProcessing {len(all_images)} valid images...")
        
        # Split dataset (70/20/10)
        np.random.seed(42)
        np.random.shuffle(all_images)
        
        train_size = int(0.7 * len(all_images))
        val_size = int(0.2 * len(all_images))
        
        train_images = all_images[:train_size]
        val_images = all_images[train_size:train_size + val_size]
        test_images = all_images[train_size + val_size:]
        
        # Process each split
        for split, images in [('train', train_images), ('val', val_images), ('test', test_images)]:
            print(f"\nProcessing {split}: {len(images)} images")
            self._process_split(images, split)
        
        # Create data.yaml
        self._create_data_yaml()
    
    def _process_split(self, image_paths, split):
        """Process images for a specific split"""
        
        for img_path in tqdm(image_paths, desc=f"Processing {split}"):
            # Copy image
            dest_img = self.output_dir / split / 'images' / img_path.name
            shutil.copy(img_path, dest_img)
            
            # Process labels
            txt_path = img_path.with_suffix('.txt')
            json_path = img_path.with_suffix('.json')
            
            with open(json_path) as f:
                json_data = json.load(f)
            
            # Handle JSON format
            if isinstance(json_data, dict) and 'pose_data' in json_data:
                pose_list = json_data['pose_data']
            else:
                pose_list = json_data
            
            # Read YOLO labels
            with open(txt_path) as f:
                yolo_lines = f.readlines()
            
            # Create enhanced labels
            new_labels = []
            
            for line in yolo_lines:
                parts = line.strip().split()
                if len(parts) >= 5:
                    old_class = int(parts[0])
                    bbox = parts[1:5]
                    
                    # Remap classes: 0,1,2 -> 0 (robot), 3 -> 1 (goal_box)
                    if old_class < 3:  # Robot
                        new_class = 0
                        
                        # Find orientation from JSON
                        robot_id = f"target_bot_{old_class + 1}"
                        yaw = 0.0
                        
                        for pose in pose_list:
                            if pose.get('robot_id') == robot_id:
                                yaw = pose.get('yaw', 0.0)
                                break
                        
                        # Store sin and cos instead of angle (better for training)
                        sin_yaw = np.sin(yaw)
                        cos_yaw = np.cos(yaw)
                        
                        # Format: class x y w h sin(yaw) cos(yaw)
                        new_labels.append(f"{new_class} {' '.join(bbox)} {sin_yaw:.6f} {cos_yaw:.6f}")
                    
                    else:  # Goal box
                        new_class = 1
                        # Goal box has no orientation
                        new_labels.append(f"{new_class} {' '.join(bbox)} 0.0 0.0")
            
            # Write new labels
            dest_label = self.output_dir / split / 'labels' / txt_path.name
            with open(dest_label, 'w') as f:
                f.write('\n'.join(new_labels))
    
    def _create_data_yaml(self):
        """Create YOLOv8 data configuration"""
        
        data_config = {
            'path': str(self.output_dir.absolute()),
            'train': 'train/images',
            'val': 'val/images', 
            'test': 'test/images',
            
            # Classes
            'names': {
                0: 'robot',  # Any jetbot
                1: 'goal_box'
            },
            'nc': 2,
            
            # Custom fields for our model
            'orientation': {
                'enabled': True,
                'classes': [0],  # Only robots have orientation
                'representation': 'sin_cos'  # Using sin/cos representation
            },
            
            # Ground-view specific settings
            'perspective': 'ground',
            'camera_height': 0.1,  # 10cm off ground
            'fov': 160  # Wide angle camera on jetbot
        }
        
        yaml_path = self.output_dir / 'data.yaml'
        with open(yaml_path, 'w') as f:
            yaml.dump(data_config, f, default_flow_style=False)
        
        print(f"\nDataset prepared at: {self.output_dir}")
        print(f"Configuration: {yaml_path}")

# Prepare the dataset
dataset_prep = GroundViewDatasetPrep(
    source_dir="C:/temp",  # Your synthetic data
    output_dir="jetbot_groundview_dataset"
)
dataset_prep.process_dataset()

Dataset Statistics:
Total images: 30000
Robot count distribution:
  0_robots: 30000 images (100.0%)

Processing 30000 valid images...

Processing train: 21000 images


Processing train: 100%|██████████| 21000/21000 [02:03<00:00, 169.93it/s]



Processing val: 6000 images


Processing val: 100%|██████████| 6000/6000 [00:48<00:00, 123.47it/s]



Processing test: 3000 images


Processing test: 100%|██████████| 3000/3000 [00:25<00:00, 119.90it/s]


Dataset prepared at: jetbot_groundview_dataset
Configuration: jetbot_groundview_dataset\data.yaml


In [ ]:
class JetsonOptimizedTrainer:
    """
    Training pipeline optimized for Jetson Nano deployment.
    Uses mixed precision, model pruning, and quantization-aware training.
    """
    
    def __init__(self, data_yaml, base_model='yolov8n.pt'):
        self.data_yaml = data_yaml
        self.base_model = base_model
        self.device = select_device('0' if torch.cuda.is_available() else 'cpu')
        
    def create_training_args(self):
        """Create training arguments optimized for ground-view detection"""
        
        args = {
            'model': self.base_model,
            'data': self.data_yaml,
            'epochs': 300,
            'patience': 50,
            'batch': 16,  # Adjust based on GPU memory
            'imgsz': 512,  # Smaller than 640 for faster inference on Jetson
            'device': self.device,
            
            # Optimization settings
            'optimizer': 'AdamW',
            'lr0': 0.01,
            'lrf': 0.01,
            'momentum': 0.937,
            'weight_decay': 0.0005,
            'warmup_epochs': 3.0,
            'warmup_momentum': 0.8,
            'warmup_bias_lr': 0.1,
            
            # Augmentation for ground-view
            'hsv_h': 0.015,  # Hue variation
            'hsv_s': 0.7,    # Saturation variation  
            'hsv_v': 0.4,    # Value variation
            'degrees': 0.0,   # No rotation (preserve ground plane)
            'translate': 0.2, # Translation
            'scale': 0.5,     # Scale variation (robots at different distances)
            'shear': 0.0,     # No shear
            'perspective': 0.0001,  # Slight perspective (ground view variation)
            'flipud': 0.0,    # No vertical flip
            'fliplr': 0.0,    # No horizontal flip (preserve left/right)
            'mosaic': 0.8,    # Mosaic augmentation
            'mixup': 0.1,     # Mixup augmentation
            
            # Loss weights
            'box': 7.5,
            'cls': 0.5,
            'dfl': 1.5,
            
            # Model optimization
            'half': True,  # FP16 training (faster, less memory)
            'cos_lr': True,  # Cosine LR scheduler
            
            # Save settings
            'save': True,
            'save_period': 20,
            'project': 'runs/train',
            'name': 'jetbot_groundview',
            'exist_ok': True,
            'pretrained': True,
            'plots': True,
            
            # Early stopping on Jetson metrics
            'val': True,
            'amp': True,  # Automatic mixed precision
        }
        
        return args
    
    def train_with_orientation(self):
        """Train model with custom orientation loss"""
        
        # Initialize model with modifications
        model = YOLO(self.base_model)
        
        # Get training args
        args = self.create_training_args()
        
        # Add custom callbacks for orientation loss
        def on_train_batch_end(trainer):
            # Log orientation loss if available
            pass
        
        # Train the model
        results = model.train(**args)
        
        return model, results
    
    def optimize_for_jetson(self, model_path):
        """Post-training optimization for Jetson deployment"""
        
        model = YOLO(model_path)
        
        # 1. Export to ONNX with optimization
        model.export(
            format='onnx',
            imgsz=512,
            half=True,  # FP16
            dynamic=False,  # Static shapes for TensorRT
            simplify=True,
            opset=12,
            batch=1  # Single image inference
        )
        
        # 2. Export to TensorRT for Jetson
        model.export(
            format='engine',
            imgsz=512,
            half=True,
            device=0,
            workspace=4,  # 4GB workspace
            batch=1
        )
        
        print("Model optimized for Jetson deployment")

# Start training
trainer = JetsonOptimizedTrainer('jetbot_groundview_dataset/data.yaml')
model, results = trainer.train_with_orientation()

# Optimize for deployment
trainer.optimize_for_jetson('runs/train/jetbot_groundview/weights/best.pt')

Testing custom YOLO wrapper...

Processing screenshot_00000.png...
  Error: 'Detect' object has no attribute 'na'

Processing screenshot_00005.png...
  Error: 'Detect' object has no attribute 'na'

Processing screenshot_00010.png...
  Error: 'Detect' object has no attribute 'na'

Processing screenshot_00015.png...
  Error: 'Detect' object has no attribute 'na'

Processing screenshot_00020.png...
  Error: 'Detect' object has no attribute 'na'

Processing screenshot_00025.png...
  Error: 'Detect' object has no attribute 'na'

Processing screenshot_00030.png...
  Error: 'Detect' object has no attribute 'na'

Processing screenshot_00035.png...
  Error: 'Detect' object has no attribute 'na'

Processing screenshot_00040.png...
  Error: 'Detect' object has no attribute 'na'

Processing screenshot_00045.png...
  Error: 'Detect' object has no attribute 'na'


Traceback (most recent call last):
  File "C:\Users\fredr\AppData\Local\Temp\ipykernel_28768\1303497928.py", line 212, in test_wrapper
    wrapper.visualize(img_path, output_path)
  File "C:\Users\fredr\AppData\Local\Temp\ipykernel_28768\1303497928.py", line 148, in visualize
    results = self.predict_with_orientation(image_path)
  File "C:\Users\fredr\AppData\Local\Temp\ipykernel_28768\1303497928.py", line 80, in predict_with_orientation
    results = self.yolo.predict(image_path, conf=conf, iou=iou, verbose=False)
  File "c:\Users\fredr\anaconda3\lib\site-packages\ultralytics\engine\model.py", line 555, in predict
    return self.predictor.predict_cli(source=source) if is_cli else self.predictor(source=source, stream=stream)
  File "c:\Users\fredr\anaconda3\lib\site-packages\ultralytics\engine\predictor.py", line 218, in __call__
    return list(self.stream_inference(source, model, *args, **kwargs))  # merge list of Result into one
  File "c:\Users\fredr\anaconda3\lib\site-packages\

In [19]:
! pip uninstall onnx -y
! pip install onnx==1.14.1


Found existing installation: onnx 1.17.0
Uninstalling onnx-1.17.0:
  Successfully uninstalled onnx-1.17.0
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.3 MB 1.4 MB/s eta 0:00:10
    --------------------------------------- 0.3/13.3 MB 3.2 MB/s eta 0:00:05
   -- ------------------------------------- 0.9/13.3 MB 7.2 MB/s eta 0:00:02
   --- ------------------------------------ 1.0/13.3 MB 7.4 MB/s eta 0:00:02
   ---- ----------------------------------- 1.6/13.3 MB 7.1 MB/s eta 0:00:02
   ------ --------------------------------- 2.1/13.3 MB 7.8 MB/s eta 0:00:02
   ------- -------------------------------- 2.5/13.3 MB 7.9 MB/s eta 0:00:02
   --------- ------------------------------ 3.1/13.3 MB 8.6 MB/s eta 0:00:02
   --------- ------------------------------ 3.1/13.3 MB 8.7 MB/s eta 0:00:02
   ---------- ----------------------------- 3.4/13.3 MB 7.4 MB/s eta 0:00:02
   ------------ --------------------------- 4.0/13.3 M

DEPRECATION: uiutil 1.38.0 has a non-standard dependency specifier future>=0.16.0attrs>=16.3.0. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of uiutil or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
def optimize_for_jetson(self, model_path):
    """Export to ONNX format on PC. TensorRT conversion must be done on Jetson."""
    model = YOLO(model_path)

    print("🔄 Exporting model to ONNX...")
    model.export(
        format='onnx',
        imgsz=512,
        half=True,
        simplify=True,
        opset=12,
        dynamic=False,
        batch=1
    )
    print("✅ Export complete. Copy the .onnx file to your Jetson for TensorRT conversion.")
trainer.optimize_for_jetson('runs/train/jetbot_groundview/weights/best.pt')


Ultralytics 8.3.141  Python-3.9.18 torch-2.5.1+cu121 CPU (AMD Ryzen 9 7950X 16-Core Processor)
WARNING half=True only compatible with GPU export, i.e. use device=0
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs\train\jetbot_groundview\weights\best.pt' with input shape (1, 3, 416, 416) BCHW and output shape(s) (1, 6, 3549) (6.0 MB)

ONNX: starting export with onnx 1.14.1 opset 12...
ONNX: slimming with onnxslim 0.1.53...
ONNX: export success  0.5s, saved as 'runs\train\jetbot_groundview\weights\best.onnx' (11.6 MB)

Export complete (0.6s)
Results saved to C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\runs\train\jetbot_groundview\weights
Predict:         yolo predict task=detect model=runs\train\jetbot_groundview\weights\best.onnx imgsz=416  
Validate:        yolo val task=detect model=runs\train\jetbot_groundview\weights\best.onnx imgsz=416 data=jetbot_groundview_dataset/data.yaml  
Visualize:       https://net

ModuleNotFoundError: No module named 'tensorrt'

In [ ]:
class JetbotGroundViewDetector:
    """
    Optimized inference for Jetson Nano with ground-view perspective.
    Handles arbitrary number of robots in frame.
    """
    
    def __init__(self, model_path, use_tensorrt=True):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        if use_tensorrt and model_path.endswith('.engine'):
            # Use TensorRT for maximum performance
            import tensorrt as trt
            import pycuda.driver as cuda
            import pycuda.autoinit
            
            self.trt_runtime = trt.Runtime(trt.Logger(trt.Logger.WARNING))
            with open(model_path, 'rb') as f:
                self.engine = self.trt_runtime.deserialize_cuda_engine(f.read())
            self.context = self.engine.create_execution_context()
            
            # Allocate buffers
            self.inputs, self.outputs, self.bindings = [], [], []
            for binding in self.engine:
                size = trt.volume(self.engine.get_binding_shape(binding))
                dtype = trt.nptype(self.engine.get_binding_dtype(binding))
                host_mem = cuda.pagelocked_empty(size, dtype)
                device_mem = cuda.mem_alloc(host_mem.nbytes)
                self.bindings.append(int(device_mem))
                if self.engine.binding_is_input(binding):
                    self.inputs.append({'host': host_mem, 'device': device_mem})
                else:
                    self.outputs.append({'host': host_mem, 'device': device_mem})
        else:
            # Use ONNX or PyTorch
            self.model = YOLO(model_path)
    
    def preprocess_groundview(self, image):
        """Preprocess image from ground-view camera"""
        # Resize to model input size
        img = cv2.resize(image, (512, 512))
        
        # Convert to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Normalize
        img = img.astype(np.float32) / 255.0
        
        # Add batch dimension and transpose to NCHW
        img = np.transpose(img, (2, 0, 1))
        img = np.expand_dims(img, 0)
        
        return img
    
    def detect_robots_and_goal(self, image, conf_threshold=0.5):
        """
        Detect all robots and goal box with orientations.
        Returns list of detections, each containing:
        - class: 'robot' or 'goal_box'
        - bbox: [x1, y1, x2, y2]
        - confidence: detection confidence
        - yaw: orientation in radians (robots only)
        - distance: estimated distance based on bbox size
        """
        
        # Preprocess
        input_img = self.preprocess_groundview(image)
        
        if hasattr(self, 'trt_runtime'):
            # TensorRT inference
            np.copyto(self.inputs[0]['host'], input_img.ravel())
            cuda.memcpy_htod(self.inputs[0]['device'], self.inputs[0]['host'])
            self.context.execute_v2(bindings=self.bindings)
            cuda.memcpy_dtoh(self.outputs[0]['host'], self.outputs[0]['device'])
            outputs = self.outputs[0]['host'].reshape(self.engine.get_binding_shape(1))
        else:
            # ONNX/PyTorch inference
            results = self.model(input_img, conf=conf_threshold)
            outputs = results[0].boxes
        
        # Parse detections
        detections = []
        
        # Process each detection
        if outputs is not None:
            for det in outputs.data:
                x1, y1, x2, y2, conf, cls = det[:6]
                
                # Get orientation if robot
                if int(cls) == 0:  # Robot class
                    sin_yaw = float(det[6])
                    cos_yaw = float(det[7])
                    yaw = np.arctan2(sin_yaw, cos_yaw)
                else:
                    yaw = None
                
                # Estimate distance from ground view
                bbox_height = y2 - y1
                # Simple distance estimation (calibrate for your camera)
                distance = 1.0 / (bbox_height / 512.0 + 0.01)
                
                detection = {
                    'class': 'robot' if int(cls) == 0 else 'goal_box',
                    'bbox': [float(x1), float(y1), float(x2), float(y2)],
                    'confidence': float(conf),
                    'yaw': yaw,
                    'distance': distance,
                    'center_x': float((x1 + x2) / 2),
                    'center_y': float((y1 + y2) / 2)
                }
                detections.append(detection)
        
        return detections
    
    def compute_navigation_command(self, detections, fov=160):
        """
        Compute navigation command based on ground-view detections.
        Handles multiple robots and finds path to goal.
        """
        
        # Find goal
        goal = None
        robots = []
        
        for det in detections:
            if det['class'] == 'goal_box':
                goal = det
            else:
                robots.append(det)
        
        if goal is None:
            # No goal visible - search by rotating
            return {
                'linear_velocity': 0.0,
                'angular_velocity': 0.5,  # Rotate to search
                'status': 'searching_for_goal'
            }
        
        # Compute angle to goal
        img_center_x = 256  # 512 / 2
        goal_pixel_offset = goal['center_x'] - img_center_x
        goal_angle = (goal_pixel_offset / img_center_x) * (fov / 2) * np.pi / 180
        
        # Check for obstacles (other robots)
        clear_path = True
        min_safe_distance = 0.5  # meters
        
        for robot in robots:
            # Check if robot is in our path
            robot_angle = ((robot['center_x'] - img_center_x) / img_center_x) * (fov / 2) * np.pi / 180
            
            if abs(robot_angle) < 0.3 and robot['distance'] < min_safe_distance:
                clear_path = False
                break
        
        # Compute velocities
        if clear_path:
            # Proportional control toward goal
            angular_vel = -2.0 * goal_angle  # Negative because image x is reversed
            linear_vel = 0.5 * (1.0 - min(abs(goal_angle), 1.0))
            status = 'moving_to_goal'
        else:
            # Obstacle avoidance
            angular_vel = 0.8 if goal_angle > 0 else -0.8
            linear_vel = 0.1
            status = 'avoiding_obstacle'
        
        return {
            'linear_velocity': float(linear_vel),
            'angular_velocity': float(angular_vel),
            'status': status,
            'goal_distance': goal['distance'],
            'num_robots_detected': len(robots)
        }
    
    def visualize_detections(self, image, detections):
        """Visualize detections with ground-view perspective indicators"""
        vis_img = image.copy()
        
        for det in detections:
            x1, y1, x2, y2 = map(int, det['bbox'])
            
            # Color based on class
            color = (0, 255, 0) if det['class'] == 'robot' else (0, 0, 255)
            
            # Draw bounding box
            cv2.rectangle(vis_img, (x1, y1), (x2, y2), color, 2)
            
            # Label with distance
            label = f"{det['class']} {det['confidence']:.2f} ({det['distance']:.1f}m)"
            cv2.putText(vis_img, label, (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
            # Draw orientation arrow for robots
            if det['yaw'] is not None:
                cx, cy = int(det['center_x']), int(det['center_y'])
                length = int((x2 - x1) * 0.4)
                
                # Arrow points in robot's facing direction
                end_x = int(cx + length * np.cos(det['yaw']))
                end_y = int(cy + length * np.sin(det['yaw']))
                
                cv2.arrowedLine(vis_img, (cx, cy), (end_x, end_y),
                               (255, 255, 0), 2, tipLength=0.3)
        
        # Draw ground perspective grid
        h, w = vis_img.shape[:2]
        for y in range(h//4, h, h//8):
            cv2.line(vis_img, (0, y), (w, y), (128, 128, 128), 1)
        
        return vis_img

# Example usage on Jetson
detector = JetbotGroundViewDetector('runs/train/jetbot_groundview/weights/best.engine')

# Process frame from camera
cap = cv2.VideoCapture(0)  # Or gstreamer pipeline for CSI camera
ret, frame = cap.read()

# Detect
detections = detector.detect_robots_and_goal(frame)

# Navigate
nav_cmd = detector.compute_navigation_command(detections)
print(f"Navigation: Linear={nav_cmd['linear_velocity']:.2f}, "
      f"Angular={nav_cmd['angular_velocity']:.2f}, "
      f"Status={nav_cmd['status']}")

# Visualize
vis_frame = detector.visualize_detections(frame, detections)
cv2.imshow('Jetbot Ground View', vis_frame)
cv2.waitKey(1)

In [ ]:
def benchmark_on_jetson(model_path, num_iterations=100):
    """Benchmark inference performance on Jetson Nano"""
    import time
    
    detector = JetbotGroundViewDetector(model_path)
    
    # Warm up
    dummy_img = np.random.randint(0, 255, (512, 512, 3), dtype=np.uint8)
    for _ in range(10):
        _ = detector.detect_robots_and_goal(dummy_img)
    
    # Benchmark
    times = []
    for _ in range(num_iterations):
        start = time.time()
        _ = detector.detect_robots_and_goal(dummy_img)
        times.append(time.time() - start)
    
    avg_time = np.mean(times) * 1000  # ms
    fps = 1000 / avg_time
    
    print(f"Inference Performance on Jetson Nano:")
    print(f"Average inference time: {avg_time:.2f} ms")
    print(f"FPS: {fps:.1f}")
    print(f"Min time: {np.min(times)*1000:.2f} ms")
    print(f"Max time: {np.max(times)*1000:.2f} ms")
    
    return fps

# Benchmark different formats
print("PyTorch model:")
benchmark_on_jetson('runs/train/jetbot_groundview/weights/best.pt')

print("\nONNX model:")
benchmark_on_jetson('runs/train/jetbot_groundview/weights/best.onnx')

print("\nTensorRT model:")
benchmark_on_jetson('runs/train/jetbot_groundview/weights/best.engine')

In [ ]:
# ROS integration example (save as jetbot_detector_node.py)
"""
#!/usr/bin/env python3

import rospy
from sensor_msgs.msg import Image, CompressedImage
from geometry_msgs.msg import Twist
from std_msgs.msg import String
from cv_bridge import CvBridge
import json

class JetbotDetectorNode:
    def __init__(self):
        rospy.init_node('jetbot_ground_detector')
        
        # Initialize detector
        model_path = rospy.get_param('~model_path', 'best.engine')
        self.detector = JetbotGroundViewDetector(model_path)
        
        # ROS setup
        self.bridge = CvBridge()
        self.image_sub = rospy.Subscriber('/camera/image_raw/compressed', 
                                         CompressedImage, 
                                         self.image_callback)
        self.cmd_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
        self.detection_pub = rospy.Publisher('/detections', String, queue_size=1)
        
        rospy.loginfo("Jetbot detector node started")
        
    def image_callback(self, msg):
        # Convert compressed image to OpenCV
        np_arr = np.frombuffer(msg.data, np.uint8)
        cv_image = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
        
        # Run detection
        detections = self.detector.detect_robots_and_goal(cv_image)
        
        # Publish detections
        det_msg = String()
        det_msg.data = json.dumps(detections)
        self.detection_pub.publish(det_msg)
        
        # Compute and publish navigation command
        nav_cmd = self.detector.compute_navigation_command(detections)
        
        twist = Twist()
        twist.linear.x = nav_cmd['linear_velocity']
        twist.angular.z = nav_cmd['angular_velocity']
        self.cmd_pub.publish(twist)
        
        rospy.loginfo(f"Detected {len(detections)} objects, "
                     f"Status: {nav_cmd['status']}")
    
    def run(self):
        rospy.spin()

if __name__ == '__main__':
    try:
        node = JetbotDetectorNode()
        node.run()
    except rospy.ROSInterruptException:
        pass
""" detector node started")
        
    def image_callback(self, msg):
        # Convert compressed image to OpenCV
        np_arr = np.frombuffer(msg.data, np.uint8)
        cv_image = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
        
        # Run detection
        detections = self.detector.detect_robots_and_goal(cv_image)
        
        # Publish detections
        det_msg = String()
        det_msg.data = json.dumps(detections)
        self.detection_pub.publish(det_msg)
        
        # Compute and publish navigation command
        nav_cmd = self.detector.compute_navigation_command(detections)
        
        twist = Twist()
        twist.linear.x = nav_cmd['linear_velocity']
        twist.angular.z = nav_cmd['angular_velocity']
        self.cmd_pub.publish(twist)
        
        rospy.loginfo(f"Detected {len(detections)} objects, "
                     f"Status: {nav_cmd['status']}")
    
    def run(self):
        rospy.spin()

if __name__ == '__main__':
    try:
        node = JetbotDetectorNode()
        node.run()
    except rospy.ROSInterruptException:
        pass
"""